Code Snippet – Live Data

This code example requests live (streaming) market data for an AAPL stock contract. 
                                                       
The IB API’s reqMktData method is used to receive real‑time tick data. 
    
Two key callback functions capture the data:

tickPrice: Prints price-related ticks, converting the tick type to a human-readable string using TickTypeEnum.toStr().

tickSize: Prints size/volume related ticks, again using TickTypeEnum.toStr() to decode the tick type. 
    
Before calling reqMktData, we also explicitly call reqMarketDataType(3) to indicate the kind of market data we want 
(in this example, “3” may correspond to delayed data, though if you’re subscribed, live data will be returned). 
    
Parameters include the generic tick list (here “232” to request the mark price) and flags for snapshot data and regulatory snapshot options.

In [1]:
from ibapi.client import EClient  # Handles communication with TWS/IB Gateway
from ibapi.wrapper import EWrapper  # Callback handlers for responses from TWS
from ibapi.contract import Contract   # Defines a financial instrument (stock, option, etc.)
from ibapi.ticktype import TickTypeEnum  # To convert tick type integers to human-readable strings

import datetime                      # Optional: Useful for time-based operations
import time                          # Required for delays to allow async operations to complete
import threading                     # To run the API's event loop in a separate thread

import logging
# Configure logging to output INFO-level messages on the console
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)


In [2]:
class TestApp(EClient, EWrapper):
    
    def __init__(self):
        """
        Initialize the TestApp, setting up both the EClient and EWrapper.
        This instance will be used to send requests and process live market data.
        """
        EClient.__init__(self, self)
        self.orderId = 0  # Starting value for our request IDs

    
    def nextValidId(self, orderId):
        """
        Called once a valid connection with TWS is established.
        Stores the initial order/request ID.
        """
        self.orderId = orderId

    
    def nextId(self):
        """
        Increment and return the next available request ID.
        Ensures that every request (live or historical) has a unique identifier.
        """
        self.orderId += 1
        return self.orderId

    
    def error(self, reqId, errorTime, errorCode, errorString, advancedOrderReject):
        """
        Callback for any errors published by TWS. Prints any errors that occur, including
        details such as reqId, error code, and error string.
        """
        print(f"Error - reqId: {reqId},  errorTime: {errorTime}, errorCode: {errorCode}, errorString: {errorString}, OrderReject: {advancedOrderReject}")

    
    def tickPrice(self, reqId, tickType, price, attrib):
        """
        Callback for receiving price ticks.
        
        Arguments:
          reqId   - The ID associated with this market data request.
          tickType- The type of tick (e.g., bid, ask, last price). It is an integer code.
          price   - The price value for the tick.
          attrib  - Attributes related to the pricing tick (optional data, e.g., past limit).
        
        The tick type is converted to a string using TickTypeEnum.toStr for clarity.
        """
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.toStr(tickType)}, price: {price}, attrib: {attrib}")

    
    def tickSize(self, reqId, tickType, size):
        """
        Callback for receiving size/volume ticks.
        
        Arguments:
          reqId   - The market data request ID.
          tickType- The type of tick indicating volume or size data.
          size    - The volume/size value for this tick.
          
        Again, the tickType is converted for a more user-friendly display.
        """
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.toStr(tickType)}, size: {size}")




In [3]:
port = 7496  # Typical port for connecting to TWS (7496 for IB Gateway live trading)
clientId = 5

# Create an instance of the TestApp and connect to TWS.
app = TestApp()
app.connect("127.0.0.1", port, clientId)
# Start the API processing loop in a separate thread so that it does not block the main thread.
threading.Thread(target=app.run).start()
time.sleep(1)  # Pause briefly to ensure a reliable connection before making requests

2025-05-03 17:10:01,288 [INFO] sent startApi
2025-05-03 17:10:01,292 [INFO] REQUEST startApi {}
2025-05-03 17:10:01,293 [INFO] SENDING startApi b'\x00\x00\x00\t\x00\x00\x00G2\x005\x00\x00'
2025-05-03 17:10:01,294 [INFO] ANSWER connectAck {}
2025-05-03 17:10:01,327 [INFO] ANSWER openOrderEnd {}
2025-05-03 17:10:01,385 [INFO] ANSWER managedAccounts {'accountsList': 'U18112846'}


Error - reqId: -1,  errorTime: 1746285027065, errorCode: 2104, errorString: Market data farm connection is OK:usfarm.nj, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027066, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027067, errorCode: 2104, errorString: Market data farm connection is OK:jfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027067, errorCode: 2104, errorString: Market data farm connection is OK:usfuture, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027067, errorCode: 2104, errorString: Market data farm connection is OK:eufarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027067, errorCode: 2104, errorString: Market data farm connection is OK:cashfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027069, errorCode: 2104, errorString: Market data farm connection is OK:eufarmnj, OrderReject: 
Error - reqId: -1,  errorTime: 1746285027069, errorCode: 2104, errorS

In [4]:
# Define a Contract for Apple Inc. stock.
mycontract = Contract()
mycontract.symbol = "AAPL"      # Ticker symbol
mycontract.secType = "STK"        # Security type: Stock
mycontract.exchange = "SMART"     # IB's SmartRouting
mycontract.currency = "USD"       # Trading currency

mycontract

2576725161680: ConId: 0, Symbol: AAPL, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:

In [5]:
# Set the market data type.
# The argument (here, 3) selects the type of market data to receive:
#  1:Live, 2:Frozen, 3:Delayed, or 3:Delayed Frozen. 
# Even if you request delayed data, if you are subscribed to live data, TWS will return live data.
app.reqMarketDataType(3)


2025-05-03 17:10:02,363 [INFO] REQUEST reqMarketDataType {'marketDataType': 3}
2025-05-03 17:10:02,364 [INFO] SENDING reqMarketDataType b'\x00\x00\x00\x08\x00\x00\x00;1\x003\x00'


In [6]:
# Request live (streaming) market data.
# Parameters:
app.reqMktData(app.nextId(), # reqId: Generated by nextId() to be unique.
               mycontract, # mycontract: The contract for which to retrieve data.
               "232", # genericTickList: "232" is used to request a specific generic tick (here, mark price).
               False, # snapshot: False means we want streaming live data rather than a single snapshot.
               False, # regulatorySnapshot: False since we are not requesting a regulatory snapshot.
               []) # mktDataOptions: [] implying no additional options are being set.

2025-05-03 17:10:02,378 [INFO] REQUEST reqMktData {'reqId': 2, 'contract': 2576725161680: ConId: 0, Symbol: AAPL, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:, 'genericTickList': '232', 'snapshot': False, 'regulatorySnapshot': False, 'mktDataOptions': []}
2025-05-03 17:10:02,379 [INFO] SENDING reqMktData b'\x00\x00\x000\x00\x00\x00\x0111\x002\x000\x00AAPL\x00STK\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x000\x00232\x000\x000\x00\x00'
2025-05-03 17:10:02,639 [INFO] ANSWER tickReqParams {'tickerId': 2, 'minTick': 0.01, 'bboExchange': '9c0001', 'snapshotPermissions': 4}
2025-05-03 17:10:02,639 [INFO] ANSWER marketDataType {'reqId': 2, 'marketDataType': 3}


Error - reqId: 2,  errorTime: 1746285028000, errorCode: 10167, errorString: Requested market data is not subscribed. Displaying delayed market data., OrderReject: 


2025-05-03 17:10:03,060 [INFO] ANSWER tickString {'reqId': 2, 'tickType': 88, 'value': '1746230393'}


reqId: 2, tickType: MARK_PRICE, price: 205.1000061, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST, price: 205.04, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST_SIZE, size: 2500
reqId: 2, tickType: DELAYED_BID, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BID_SIZE, size: 0
reqId: 2, tickType: DELAYED_ASK, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_ASK_SIZE, size: 0
reqId: 2, tickType: DELAYED_OPEN, price: 0.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
Error - reqId: -1,  errorTime: 1746285220362, errorCode: 2103, errorString: Market data farm connection is broken:hfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285222543, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746285682277, errorCode: 2103, errorString: Market data farm 

2025-05-03 17:35:34,605 [INFO] ANSWER tickString {'reqId': 2, 'tickType': 88, 'value': '1746230393'}


...

reqId: 2, tickType: DELAYED_LAST, price: 205.04, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST_SIZE, size: 2500
reqId: 2, tickType: DELAYED_HIGH, price: 206.99, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LOW, price: 202.16, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_VOLUME, size: 1014125
reqId: 2, tickType: DELAYED_CLOSE, price: 213.32, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
Error - reqId: -1,  errorTime: 1746286910727, errorCode: 2104, errorString: Market data farm connection is OK:usfarm.nj, OrderReject: 
Error - reqId: -1,  errorTime: 1746286913973, errorCode: 2106, errorString: HMDS data farm connection is OK:fundfarm, OrderReject: 
reqId: 2, tickType: DELAYED_BID, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BID_SIZE, size: 0
reqId: 2, tickType: DELAYED_ASK, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0

2025-05-03 22:56:05,273 [INFO] ANSWER tickString {'reqId': 2, 'tickType': 88, 'value': '1746230393'}


reqId: 2, tickType: DELAYED_HIGH, price: 206.99, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LOW, price: 202.16, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_VOLUME, size: 1014125
reqId: 2, tickType: DELAYED_CLOSE, price: 213.32, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST, price: 205.04, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST_SIZE, size: 2500
Error - reqId: -1,  errorTime: 1746305764609, errorCode: 2104, errorString: Market data farm connection is OK:cashfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746305754457, errorCode: 2105, errorString: HMDS data farm connection is broken:fundfarm, OrderReject: 
reqId: 2, tickType: DELAYED_OPEN, price: 0.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BID, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BI

2025-05-03 23:01:16,557 [INFO] ANSWER tickString {'reqId': 2, 'tickType': 88, 'value': '1746230393'}


reqId: 2, tickType: DELAYED_HIGH, price: 206.99, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LOW, price: 202.16, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_VOLUME, size: 1014125
reqId: 2, tickType: DELAYED_CLOSE, price: 213.32, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST, price: 205.04, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_LAST_SIZE, size: 2500
reqId: 2, tickType: DELAYED_OPEN, price: 0.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BID, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_BID_SIZE, size: 0
reqId: 2, tickType: DELAYED_ASK, price: -1.0, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 2, tickType: DELAYED_ASK_SIZE, size: 0
